In [1]:
import math
import random
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data as mnist_data

In [2]:
def forward_propagation_for_predict(X, parameters):
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    W4 = parameters['W4']
    W5 = parameters['W5']
    b1 = parameters['b1']
    b2 = parameters['b2']
    b3 = parameters['b3']
    b4 = parameters['b4']
    b5 = parameters['b5']
    
    Z1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = "SAME")
    A1 = tf.nn.relu(Z1 + b1)
    Z2 = tf.nn.conv2d(A1, W2, strides = [1,2,2,1], padding = "SAME")
    A2 = tf.nn.relu(Z2 + b2)
    Z3 = tf.nn.conv2d(A2, W3, strides = [1,2,2,1], padding = "SAME")
    A3 = tf.nn.relu(Z3 + b3)    
    P1 = tf.contrib.layers.flatten(A3)
    Z4 = tf.add(tf.matmul(P1, W4), b4)
    A4 = tf.nn.relu(Z4)
    Z5 = tf.matmul(A4, W5) + b5
    return Z5

In [3]:
def imageprepare(argv):
    """
    This function returns the pixel values.
    The imput is a png file location.
    """
    im = Image.open(argv).convert('L')
    width = float(im.size[0])
    height = float(im.size[1])
    newImage = Image.new('L', (28, 28), (255))  # creates white canvas of 28x28 pixels

    if width > height:  # check which dimension is bigger
        # Width is bigger. Width becomes 20 pixels.
        nheight = int(round((20.0 / width * height), 0))  # resize height according to ratio width
        if (nheight == 0):  # rare case but minimum is 1 pixel
            nheight = 1
            # resize and sharpen
        img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wtop = int(round(((28 - nheight) / 2), 0))  # calculate horizontal position
        newImage.paste(img, (4, wtop))  # paste resized image on white canvas
    else:
        # Height is bigger. Heigth becomes 20 pixels.
        nwidth = int(round((20.0 / height * width), 0))  # resize width according to ratio height
        if (nwidth == 0):  # rare case but minimum is 1 pixel
            nwidth = 1
            # resize and sharpen
        img = im.resize((nwidth, 20), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wleft = int(round(((28 - nwidth) / 2), 0))  # caculate vertical pozition
        newImage.paste(img, (wleft, 4))  # paste resized image on white canvas

    # newImage.save("sample.png

    tv = list(newImage.getdata())  # get pixel values

    # normalize pixels to 0 and 1. 0 is pure white, 1 is pure black.
    tva = [(255 - x) * 1.0 / 255.0 for x in tv]
    return tva

In [4]:
# Download images and labels into mnist.test (10K images+labels) and mnist.train (60K images+labels)
mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)
print ("mnist.train.images shape" + str(mnist.train.images.shape))
print ("mnist.test.images shape" + str(mnist.test.images.shape))
print ("mnist.train.images labels shape" + str(mnist.train.labels.shape))
print ("mnist.test.images labels shape" + str(mnist.test.labels.shape))

Extracting data\train-images-idx3-ubyte.gz
Extracting data\train-labels-idx1-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz
mnist.train.images shape(60000, 28, 28, 1)
mnist.test.images shape(10000, 28, 28, 1)
mnist.train.images labels shape(60000, 10)
mnist.test.images labels shape(10000, 10)


In [5]:
x_train = np.array(mnist.train.images)
x_test = np.array(mnist.test.images)

y_train = np.array(mnist.train.labels) 
y_test = np.array(mnist.test.labels)

print("Train dataset shape = " + str(x_train.shape))
print("Test dataset shape = " + str(x_test.shape))

print("Train dataset Labels shape = " + str(y_train.shape))
print("Test dataset Labels shape = " + str(y_test.shape))

Train dataset shape = (60000, 28, 28, 1)
Test dataset shape = (10000, 28, 28, 1)
Train dataset Labels shape = (60000, 10)
Test dataset Labels shape = (10000, 10)


In [6]:
#Normalizing the dataset
x_train/=255
x_test/=255

In [7]:
ops.reset_default_graph()
n_y = y_train.shape[1]
#Declaring Placeholders
X = tf.placeholder(tf.float32, shape=[None, 28, 28, 1], name='X') #As the image is 28*28*1
Y = tf.placeholder(tf.float32, shape=[None, n_y], name='Y') #n_y is number of classes

In [8]:
#Variable Initialization
W1 = tf.get_variable("W1", [5, 5, 1, 4], initializer=tf.contrib.layers.xavier_initializer(seed=1)) # 5x5 patch, 1 input channel, 4 output channels
b1 = tf.get_variable("b1", [4], initializer=tf.zeros_initializer())
W2 = tf.get_variable("W2", [5, 5, 4, 8], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b2 = tf.get_variable("b2", [8], initializer=tf.zeros_initializer())
W3 = tf.get_variable("W3", [4, 4, 8, 12], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b3 = tf.get_variable("b3", [12], initializer=tf.zeros_initializer())

W4 = tf.get_variable("W4", [7*7*12, 100], initializer=tf.contrib.layers.xavier_initializer(seed=1))
b4 = tf.get_variable("b4", [100], initializer=tf.zeros_initializer())
W5 = tf.get_variable("W5", [100, 10], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.get_variable("b5", [10], initializer=tf.zeros_initializer())
#Define a dictionary with parameters
parameters = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3,
              "W4": W4,
              "b4": b4,
              "W5": W5,
              "b5": b5}

In [9]:
#Model Creation and Forward Propogation
Z1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = "SAME") #Output is 28*28
A1 = tf.nn.relu(tf.add(Z1, b1))
Z2 = tf.nn.conv2d(A1, W2, strides = [1,2,2,1], padding = "SAME") # output is 14x14
A2 = tf.nn.relu(tf.add(Z2, b2))
Z3 = tf.nn.conv2d(A2, W3, strides = [1,2,2,1], padding = "SAME") # output is 7x7
A3 = tf.nn.relu(tf.add(Z3, b3))
# Flatten the output for Fully connected layer
P1 = tf.contrib.layers.flatten(A3)
Z4 = tf.add(tf.matmul(P1, W4), b4)
A4 = tf.nn.relu(Z4)
Z5 = tf.add(tf.matmul(A4, W5), b5)
#Z4 = tf.contrib.layers.fully_connected(P1, num_outputs = 10, activation_fn=None)
print("Z5 =", str(Z5))

Z5 = Tensor("Add_4:0", shape=(?, 10), dtype=float32)


In [10]:
#Cost Computation
logits = Z5
labels = Y
predict_y = tf.nn.softmax(logits)
print ("Shape Z5  =", str(logits))
print ("Shape Y  =", str(labels))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
print("Cost =", str(cost))

Shape Z5  = Tensor("Add_4:0", shape=(?, 10), dtype=float32)
Shape Y  = Tensor("Y:0", shape=(?, 10), dtype=float32)
Cost = Tensor("Mean:0", shape=(), dtype=float32)


In [11]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost) #Changed Learning Rate
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(5):
        for minibatch in range(600):
            epoch_cost = 0.                       # Defines a cost related to an epoch
            # Select a minibatch
            batch_X, batch_Y = mnist.train.next_batch(100)
            _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: batch_X, Y: batch_Y}) 
            epoch_cost += minibatch_cost / 600
        if epoch % 1 == 0:
            print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
    # lets save the parameters in a variable
    parameters = sess.run(parameters)
    print ("Parameters have been trained!")
    #Identify Correct Predictions
    correct_prediction = tf.equal(tf.argmax(predict_y, 1), tf.argmax(Y, 1)) #Z5 should we replace? = tf.nn.softmax(Z5)
    #Calculate Accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print ("Train Accuracy:", accuracy.eval({X: x_train, Y: y_train}))
    print ("Test Accuracy:", accuracy.eval({X: x_test, Y: y_test}))

Cost after epoch 0: 0.000536
Cost after epoch 1: 0.000541
Cost after epoch 2: 0.000433
Cost after epoch 3: 0.000271
Cost after epoch 4: 0.000306
Parameters have been trained!
Train Accuracy: 0.09035
Test Accuracy: 0.0892


In [ ]:
#Prediction on Own Image

import scipy
from PIL import Image, ImageFilter
from scipy import ndimage
from matplotlib import pyplot as plt

#Image Processing to fit the algorithm
my_image_logic = [imageprepare('images/handwriting.png')]
my_image = np.transpose(my_image_logic)
newArr=[[0 for d in range(28)] for y in range(28)]
k = 0
for i in range(28):
    for j in range(28):
        newArr[i][j]=my_image_logic[0][k]
        k=k+1
plt.imshow(newArr, interpolation='nearest')
plt.show()
i = np.array(newArr)
i = i.reshape(1, 28, 28, 1)
W1 = tf.convert_to_tensor(parameters["W1"])
W2 = tf.convert_to_tensor(parameters["W2"])
W3 = tf.convert_to_tensor(parameters["W3"])
W4 = tf.convert_to_tensor(parameters["W4"])
W5 = tf.convert_to_tensor(parameters["W5"])
b1 = tf.convert_to_tensor(parameters["b1"])
b2 = tf.convert_to_tensor(parameters["b2"])
b3 = tf.convert_to_tensor(parameters["b3"])
b4 = tf.convert_to_tensor(parameters["b4"])
b5 = tf.convert_to_tensor(parameters["b5"])

    
params = {"W1": W1,
          "b1": b1,
          "W2": W2,
          "b2": b2,
          "W3": W3,
          "b3": b3,
          "W4": W4,
          "b4": b4,
          "W5": W5,
          "b5": b5}

x = tf.placeholder(tf.float32, [1, 28, 28, 1])

z5 = forward_propagation_for_predict(x, params)

p = tf.argmax(z5, 1)

sess = tf.Session()
my_image_prediction = sess.run(p, feed_dict = {x: i})

print("Your algorithm predicts: y = " + str(np.squeeze(my_image_prediction)))

In [ ]:
print (str(parameters[""]))